In [19]:
API_KEY = 'a6af2051-a60d-4d5b-ae19-994ff9f22f24_up9205w'

In [20]:
import requests
import json

class Smartlead:
    BASE_URL = 'https://server.smartlead.ai/api/v1'

    def __init__(self, api_key):
        self.api_key = api_key

    # add create campaign method

    def get_campaign_sequences(self, campaign_id):
        url = f"{self.BASE_URL}/campaigns/{campaign_id}/sequences?api_key={self.api_key}"
        response = requests.get(url)
        return response.json()

    def get_campaign(self, campaign_id):
        url = f"{self.BASE_URL}/campaigns/{campaign_id}?api_key={self.api_key}"
        response = requests.get(url)
        return response.json()

    def get_campaign_email_accounts(self, campaign_id):
        url = f"{self.BASE_URL}/campaigns/{campaign_id}/email-accounts?api_key={self.api_key}"
        response = requests.get(url)
        return response.json()

    def get_campaign_leads(self, campaign_id, offset=0, limit=10):
        url = f"{self.BASE_URL}/campaigns/{campaign_id}/leads?api_key={self.api_key}&offset={offset}&limit={limit}"
        response = requests.get(url)
        return response.json()
    
    def post_campaign_leads(self, campaign_id, lead_list):
        """`lead_list` format is 
            
        lead_list = [
            {
                "first_name": "Cristiano",
                "last_name": "Ronaldo",
                "email": "cristiano@mufc.com",
                "phone_number": "0239392029",
                "company_name": "Manchester United",
                "website": "mufc.com",
                "location": "London",
                "custom_fields": {
                    "Title": "Regional Manager",
                    "First_Line": "Loved your recent post about remote work on Linkedin"
                },
                "linkedin_profile": "http://www.linkedin.com/in/cristianoronaldo",
                "company_url": "mufc.com"
            }
            # ... (add up to 100 leads)
        ]
        """
        settings = {
            "ignore_global_block_list": True,
            "ignore_unsubscribe_list": True,
            "ignore_duplicate_leads_in_other_campaign": False
        }
        if not isinstance(lead_list, list) or not isinstance(settings, dict):
            raise ValueError("lead_list must be a list and settings must be a dictionary.")

        if len(lead_list) > 100:
            raise ValueError("You can only send a maximum of 100 leads at a time.")

        url = f"{self.BASE_URL}/campaigns/{campaign_id}/leads?api_key={self.api_key}"
        data = {
            "lead_list": lead_list,
            "settings": settings
        }
        headers = {'Content-Type': 'application/json'}
        response = requests.post(url, headers=headers, data=json.dumps(data))
        return response.json()

# Example usage:
api_key = API_KEY
campaign_id = '123383'

smartlead = Smartlead(api_key)
sequences = smartlead.get_campaign_sequences(campaign_id)
# campaign = smartlead.get_campaign(campaign_id)
# email_accounts = smartlead.get_campaign_email_accounts(campaign_id)
# leads = smartlead.get_campaign_leads(campaign_id, offset=0, limit=10)

print(sequences)
# print(campaign)
# print(email_accounts)
# print(leads)


[{'id': 387060, 'created_at': '2023-10-27T18:11:13.050Z', 'updated_at': '2023-10-27T18:11:13.050Z', 'email_campaign_id': 123383, 'seq_number': 1, 'seq_delay_details': {'delayInDays': 1}, 'subject': '{{subject_line}}', 'email_body': '<div>{{body}}&nbsp;</div>', 'sequence_variants': None}]


In [21]:
lead_list = [
    {
        "first_name": "Cristiano",
        "last_name": "Ronaldo",
        "email": "cristiano@mufc.com",
        "phone_number": "0239392029",
        "company_name": "Manchester United",
        "website": "mufc.com",
        "location": "London",
        "custom_fields": {
            "Title": "Regional Manager",
            "First_Line": "Loved your recent post about remote work on Linkedin"
        },
        "linkedin_profile": "http://www.linkedin.com/in/cristianoronaldo",
        "company_url": "mufc.com"
    }
    # ... (add up to 100 leads)
]



response = smartlead.post_campaign_leads(campaign_id, lead_list)


In [22]:

CAMPAIGN_STATS_URL = f"https://server.smartlead.ai/api/v1/campaigns/132424/statistics?api_key={API_KEY}"

response = requests.get(CAMPAIGN_STATS_URL)
response = response.json()

data = response['data']

sent_count = 0
open_count = 0
reply_count = 0
for item in data:
    if item.get('sent_time'):
        sent_count += 1

    if item.get('open_time'):
        open_count += 1

    if item.get('reply_time'):
        reply_count += 1

print(f"Sent: {sent_count}")
print(f"Open: {open_count}")
print(f"Reply: {reply_count}")

print(data)

Sent: 177
Open: 47
Reply: 9
[{'lead_name': 'John ', 'lead_email': 'john.cullen@castandcrew.com', 'lead_category': None, 'sequence_number': 1, 'stats_id': '00ee3ec7-15e8-4d72-a028-b14f1ff51a09', 'email_campaign_seq_id': 415534, 'seq_variant_id': 224889, 'email_subject': 'fellow ucsc alum - small ask', 'email_message': "<div>Hi&nbsp;John,</div><div><br></div><div>I’m a fellow Slug reaching out with a small ask - would you be willing to loop me in with whoever oversees application security and config at Cast & Crew?</div><div><br></div><div>My company helps developers and security teams synchronize their secrets and app configuration across various devices, environments, and team members. We're bringing a new approach to handling .env files, helping with efficiency and security.</div><div><br></div><div>I'd love the opportunity to assist a fellow alum and everyone at Cast & Crew!</div><div><br></div><div>- Adam</div>", 'sent_time': '2023-11-23T18:03:06.029Z', 'open_time': None, 'click_tim